In [36]:
%run dados.ipynb

In [37]:
spark = (SparkSession.builder
        .master("local")
        .appName("projetosoulcode")
        .config('spark.jars','gcs-connector-hadoop3-latest.jar' )
        .config('spark.hadoop.fs.gs.impl','com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')
        .config('spark.hadoop.fs.gs.auth.service.account.enable', 'true')
        .config("google.cloud.auth.service.account.json.keyfile",'soulcodeproject-44ce61622773.json')
        .config('spark.hadoop.fs.gs.project.id','soulcodeproject')
        .getOrCreate())
spark.conf.set("spark.sql.caseSensitive", "true")

In [38]:
esquema = (
    StructType([
        StructField("ID",IntegerType(), False),
        StructField("Ano_Nascimento", IntegerType(), True),
        StructField("GrauInstrução", StringType(), True),
        StructField("EstadoCivil", StringType(), True),
        StructField("Renda", DoubleType(), True),
        StructField("NumCrianca", IntegerType(), True),
        StructField("NumAdolescente", IntegerType(), True),
        StructField("DataConta", StringType(), True),
        StructField("UltimaCompra", IntegerType(), True),
        StructField("QtdVinho", IntegerType(), True),
        StructField("QtdFrutas", IntegerType(), True),
        StructField("QtdCarne", IntegerType(), True),
        StructField("QtdPeixe", IntegerType(), True),
        StructField("QtdDoces", IntegerType(), True),
        StructField("QtdOuro", IntegerType(), True),
        StructField("QtdComprasDesconto", IntegerType(), True),
        StructField("QtdComprasWeb", IntegerType(), True),
        StructField("QtdCompraCatalogo", IntegerType(), True),
        StructField("QtdCompraLoja", IntegerType(), True),
        StructField("NumVisitasSite", IntegerType(), True),
        StructField("AceitouCam3", IntegerType(), True),
        StructField("AceitouCam4", IntegerType(), True),
        StructField("AceitouCam5", IntegerType(), True),
        StructField("AceitouCam1", IntegerType(), True),
        StructField("AceitouCam2", IntegerType(), True),
        StructField("Reclamacao", IntegerType(), True),
        StructField("CustoContato", IntegerType(), True),
        StructField("Receita", IntegerType(), True),
        StructField("UltimaCampanha", IntegerType(), True),
    ])
)

In [39]:
df2= (
    spark
       .read
       .format("csv")
       .option("header", "true")
       .option("delimiter", ",")
       .load('gs://soulcodeprojeto/marketing_campaign.csv', schema=esquema)
       .withColumnRenamed('Year_Birth', 'AnoNascimento')
)

In [41]:
pandasDF = df2.toPandas()

22/03/18 17:00:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, Year_Birth, Education, Marital_Status, Income, Kidhome, Teenhome, Dt_Customer, Recency, MntWines, MntFruits, MntMeatProducts, MntFishProducts, MntSweetProducts, MntGoldProds, NumDealsPurchases, NumWebPurchases, NumCatalogPurchases, NumStorePurchases, NumWebVisitsMonth, AcceptedCmp3, AcceptedCmp4, AcceptedCmp5, AcceptedCmp1, AcceptedCmp2, Complain, Z_CostContact, Z_Revenue, Response
 Schema: ID, Ano_Nascimento, GrauInstrução, EstadoCivil, Renda, NumCrianca, NumAdolescente, DataConta, UltimaCompra, QtdVinho, QtdFrutas, QtdCarne, QtdPeixe, QtdDoces, QtdOuro, QtdComprasDesconto, QtdComprasWeb, QtdCompraCatalogo, QtdCompraLoja, NumVisitasSite, AceitouCam3, AceitouCam4, AceitouCam5, AceitouCam1, AceitouCam2, Reclamacao, CustoContato, Receita, UltimaCampanha
Expected: Ano_Nascimento but found: Year_Birth
CSV file: gs://soulcodeprojeto/marketing_campaign.csv


In [44]:
pandasDF.head()

,ID,Ano_Nascimento,GrauInstrução,EstadoCivil,Renda,NumCrianca,NumAdolescente,DataConta,UltimaCompra,QtdVinho,...,NumVisitasSite,AceitouCam3,AceitouCam4,AceitouCam5,AceitouCam1,AceitouCam2,Reclamacao,CustoContato,Receita,UltimaCampanha
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0
